# VEGAS in spinfoams: application on a toy model

In [1]:
using JupyterFormatter
enable_autoformat()

1-element Vector{Function}:
 format_current_cell (generic function with 1 method)

This toy model has the purpose of testing the VEGAS algorithm in order to compute a discret sum.

I consider a function $F(x_0, y_0, z_0)$ of 3 variables defined as:

\begin{align}
F(x_0, y_0, z_0) & = \sum\limits_{x = 0}^{10} \sum\limits_{y = 0}^{10} \sum\limits_{z = 0}^{10} e^{-\frac{1}{2} \left( x - x_0 \right)^2} e^{-\frac{1}{2} \left( y - y_0 \right)^2} e^{-\frac{1}{2} \left( z - z_0 \right)^2} \sin \left( xy \right) \cos \left( z \right) \\
& \equiv \sum\limits_{x = 0}^{10} \sum\limits_{y = 0}^{10} \sum\limits_{z = 0}^{10} f(x_0, y_0, z_0, x, y, z)
\end{align}

In [1]:
function F(x0, y0, z0)

    result = 0.0

    for x = 0:10, y = 0:10, z = 0:10
        result +=
            exp(-(1 / 2) * (x - x0)^2) *
            exp(-(1 / 2) * (y - y0)^2) *
            exp(-(1 / 2) * (z - z0)^2) *
            sin(x * y) *
            cos(z)
    end

    result

end

F (generic function with 1 method)

## Exact result

Let's first compute the exact value of the function for a chosen value of $(x_0, y_0, z_0)$:

In [2]:
@time F(2, 6, 10)

  0.000045 seconds


0.692056321795933

## Standard Monte Carlo

Let's now construct a standard Monte Carlo estimator for the above function

$$
\lim_{N\to\infty} F_{MC}(x_0, y_0, z_0, N) = F(x_0, y_0, z_0) 
$$

which is given by:

\begin{align}
F_{MC}(x_0, y_0, z_0, N) & = \frac{V}{N} \sum\limits_{[x, y, z]_1 \dots [x, y, z]_N} e^{-\frac{1}{2} \left( x - x_0 \right)^2} e^{-\frac{1}{2} \left( y - y_0 \right)^2} e^{-\frac{1}{2} \left( z - z_0 \right)^2} \sin \left( xy \right) \cos \left( z \right) \\
& \equiv \frac{V}{N} \sum\limits_{[x, y, z]_1 \dots [x, y, z]_N} f(x_0, y_0, z_0, [x, y, z])
\end{align}

where $ V = 11^3 $, as this is the total number of possible configurations. 

With respect to the discrete case, in the continuous $V$ coincides with the volume of the subset of $\mathbb{R^3}$ on which the integration is performed.

In [3]:
using Distributions, Random

In [33]:
function FMC(x0, y0, z0, N, V)

    draw = Array{Int}(undef, 3)
    draw_float_sample = Array{Float64}(undef, 1)
    Uniform_distribution = Uniform(0, 11)

    result = 0.0

    for i = 1:N

        for i = 1:3
            rand!(Uniform_distribution, draw_float_sample)
            draw_float_sample[1] = floor(draw_float_sample[1])
            draw[i] = Int(draw_float_sample[1])
        end

        result +=
            exp(-(1 / 2) * (draw[1] - x0)^2) *
            exp(-(1 / 2) * (draw[2] - y0)^2) *
            exp(-(1 / 2) * (draw[3] - z0)^2) *
            sin(draw[1] * draw[2]) *
            cos(draw[3])

    end

    result *= V / N

    println(result)

end

FMC (generic function with 1 method)

In [34]:
V = 11^3;

In [35]:
for i=1:10

@time FMC(2, 6, 10, 10^7, V)
    
end

0.6901471653105549
  0.893630 seconds (198.51 k allocations: 11.399 MiB, 9.02% compilation time)
0.699621777116541
  0.761710 seconds (29 allocations: 1.453 KiB)
0.7040484902062966
  0.760316 seconds (28 allocations: 1.062 KiB)
0.6922514473029477
  0.758617 seconds (28 allocations: 1.062 KiB)
0.721554044764387
  0.772173 seconds (28 allocations: 1.062 KiB)
0.6753680313835795
  0.766042 seconds (29 allocations: 2.047 KiB)
0.6860834314683691
  0.778286 seconds (28 allocations: 1.062 KiB)
0.7053147339883762
  0.755483 seconds (28 allocations: 1.062 KiB)
0.7026439691337742
  0.747401 seconds (28 allocations: 1.062 KiB)
0.6910096170906878
  0.742870 seconds (28 allocations: 1.062 KiB)


- $\textbf{The converge is slow}$: we need $10^7$ iterations to see a result stable up to the first significant digit. 

- There is a $\textbf{very high standard deviation}$ (see next cell)

- $\textbf{The converges is biased}$, since the sum is highly oscillating and has several cancellations due to the negative terms.

### Standard deviation

The standard deviations $\sigma^2_{F}$ $\sigma^2_{F_{MC}}$ (of $F$ and $F_{MC}$ respectively) are an indication of the possible error in the Monte Carlo estimate. 

Let's start by defining 

$$ 
\langle F \rangle \equiv \frac{1}{N} \sum\limits_{[x, y, z]_1 \dots [x, y, z]_N} f(x_0, y_0, z_0, [x, y, z]) = \frac{1}{V}F_{MC}(x_0, y_0, z_0, N)
$$

Variance $\sigma^2_{F}$ can be estimated from the Monte Carlo data as:

\begin{align}
\sigma^2_{F} & \equiv \frac{1}{N-1} \sum\limits_{[x, y, z]_1 \dots [x, y, z]_N} \left( f(x_0, y_0, z_0, [x, y, z]) - \langle F \rangle \right)^2 \\
 & =  \frac{1}{N-1} \left[ \sum\limits_{[x, y, z]_1 \dots [x, y, z]_N} f(x_0, y_0, z_0, [x, y, z])^2 - N \langle F \rangle^2 \right] \\ 
\end{align}


From which $\sigma^2_{F_{MC}}$ is given by:

$$
\sigma^2_{MC} = \frac{V^2}{N} \sigma^2_{F} = \frac{V^2}{N-1} \left[ \frac{1}{N} \sum\limits_{[x, y, z]_1 \dots [x, y, z]_N} f(x_0, y_0, z_0, [x, y, z])^2 - \langle F \rangle^2 \right]
$$

# Changing variables

I can perform a trivial change of variables in order to sample from the interval $[0,1]$:

$$
X = \frac{x}{10}, \ Y = \frac{y}{10},\  Z = \frac{z}{10}
$$

The function changes as:

$$
F(x_0, y_0, z_0) = \sum\limits_{X \in [0,1]} \sum\limits_{Y \in [0,1]} \sum\limits_{Z \in [0,1]} f(x_0, y_0, z_0, x(X), y(Y), z(Z))
$$

Since the original quantity was a sum and not a continuous integral, of course it is necessary to round.

The MC estimator becomes:

\begin{align}
F_{MC}(x_0, y_0, z_0, N) & = \frac{V}{N} \sum\limits_{[X, Y, Z]_1 \dots [X, Y, Z]_N} f(x_0, y_0, z_0, [x(X), y(Y), z(Z)])
\end{align}

In [20]:
function FMC_change_variable(x0, y0, z0, N, V)

    draw = Array{Int}(undef, 3)
    draw_float_sample = Array{Float64}(undef, 1)
    Uniform_distribution = Uniform(0, 1)

    result = 0.0

    for i = 1:N

        for i = 1:3
            rand!(Uniform_distribution, draw_float_sample)
            draw_float_sample[1] = 10 * draw_float_sample[1]
            draw_float_sample[1] = round(Int64, draw_float_sample[1])
            draw[i] = Int(draw_float_sample[1])
        end

        result +=
            exp(-(1 / 2) * (draw[1] - x0)^2) *
            exp(-(1 / 2) * (draw[2] - y0)^2) *
            exp(-(1 / 2) * (draw[3] - z0)^2) *
            sin(draw[1] * draw[2]) *
            cos(draw[3])

    end

    result *= V / N

    result

end

FMC_change_variable (generic function with 1 method)

In [76]:
@time FMC_change_variable(2, 6, 10, 10^7, V)

  0.765712 seconds (80.87 k allocations: 4.597 MiB, 3.64% compilation time)


0.6512009430339419

## Introducing a Jacobian

We can rewrite the above expression by introducing the following Jacobian for the coordinate transformation:

$$
J (X,Y,Z) = 
\begin{vmatrix}
\frac{d x}{d X} & 0 & 0 \\
0 & \frac{d y}{d Y} & 0 \\
0 & 0 & \frac{d z}{d Z} \\
\end{vmatrix}
=
10^3
$$

which leads to:

\begin{align}
F_{MC}(x_0, y_0, z_0, N) = \frac{V_J}{N} \sum\limits_{[X, Y, Z]_1 \dots [X, Y, Z]_N} J \cdot f(x_0, y_0, z_0, [x(X), y(Y), z(Z)])
\label{MC_estimator} \tag{1}
\end{align}

In the continuous case we have $V_J = 1$.

$$ 
\langle F \rangle_J \equiv \frac{1}{N} \sum\limits_{[X, Y, Z]_1 \dots [X, Y, Z]_N} J \cdot f(x_0, y_0, z_0, [x, y, z]) = \frac{1}{V_J}F_{MC}(x_0, y_0, z_0, N)
$$

Variance $\sigma^2_{F_J}$ is estimated as:

\begin{align}
\sigma^2_{F_J} & \equiv \frac{1}{N-1} \sum\limits_{[X, Y, Z]_1 \dots [X, Y, Z]_N} \left( J \cdot f(x_0, y_0, z_0, [x(X), y(Y), z(Z)]) - \langle F \rangle_J \right)^2 \\
 & =  \frac{1}{N-1} \left[ \sum\limits_{[X, Y, Z]_1 \dots [X, Y, Z]_N} J^2 \cdot f(x_0, y_0, z_0, [x(X), y(Y), z(Z)])^2 - N \langle F \rangle_J^2 \right] \\ 
\end{align}


Variance $\sigma^2_{F_{MC_J}}$ is estimated as:

$$
\sigma^2_{MC_J} = \frac{V_J^2}{N} \sigma^2_{F_J} = \frac{V_J^2}{N-1} \left[ \frac{1}{N} \sum\limits_{[X, Y, Z]_1 \dots [X, Y, Z]_N} J^2 \cdot f(x_0, y_0, z_0, [x(X), y(Y), z(Z)])^2 - \langle F \rangle_J^2 \right]
$$

## VEGAS

I write explicitly the equations considering a single axis for simplicity. 

For each axis, we have a grid with $a = x_0$, $b = x_{N_g}$, where $N_g$ is the total number of elements in the grid. 

$ x_0 = a $

$ x_1 = x_0 + \Delta x_0 $

$ x_2 = x_1 + \Delta x_1 $

$ \dots $

$ x_{N_g} = x_{N_g-1} + \Delta x_{N_g-1} = b $

Let's define a matrix which contains the grid over the 3 axes:

\begin{align}
\text{VEGAS_grid}^{T} = 
\begin{pmatrix}
x_0 & x_1 & \dots & x_{N_g} \\
y_0 & y_1 & \dots & y_{N_g}  \\
z_0 & z_1 & \dots & z_{N_g}  \\
\end{pmatrix}
\end{align}

and matrix for the corresponding spacing:

\begin{align}
\text{VEGAS_spacing_grid}^{T} = 
\begin{pmatrix}
\Delta x_0 & \Delta x_1 & \dots & \Delta x_{N_g-1} \\
\Delta y_0 & \Delta y_1 & \dots & \Delta y_{N_g-1}  \\
\Delta z_0 & \Delta z_1 & \dots & \Delta z_{N_g-1}  \\
\end{pmatrix}
\end{align}

The first natural choice for the grid is a linear spaced one. We therefore consider initially all equal $\Delta x_i = x_{i+1} - x_{i} = \frac{b-a}{N_g}$ for $i = 0 \dots N_g -1$.  

In [23]:
a = 0;
b = 10;
N_g = 25;
N_ev = 10^7

VEGAS_grid = Array{Float64}(undef, N_g + 1, 3);
VEGAS_grid_improved = Array{Float64}(undef, N_g + 1, 3);

for i = 1:3
    VEGAS_grid[:, i] .= collect(LinRange(a, b, N_g + 1))
end

#Delta_x_i = (b - a) / (N_g);

VEGAS_spacing_grid = Array{Float64}(undef, N_g, 3);
VEGAS_spacing_grid_improved = Array{Float64}(undef, N_g, 3);

#for i = 1:3
#    VEGAS_spacing_grid[:, i] .= [Delta_x_i for i = 1:1:N_g]
#end

for i = 1:3
    for j = 1:N_g
        VEGAS_spacing_grid[j, i] = VEGAS_grid[j+1, i] - VEGAS_grid[j, i]
    end
end

In [24]:
VEGAS_grid;

In [25]:
VEGAS_spacing_grid;

The following function provides the map between $X \in [0,1]$ and $x \in [a, b]$

In [22]:
# Returns the VEGAS grid element and the corresponding grid index
function x(X, N_g, VEGAS_grid, VEGAS_spacing_grid)

    i_X = Int(floor(X * N_g))

    delta_X = X * N_g - i_X

    if (i_X == N_g)
        VEGAS_grid[i_X+1, 1], i_X + 1
    else
        VEGAS_grid[i_X+1, 1] + VEGAS_spacing_grid[i_X+1, 1] * delta_X, i_X + 1
    end

end


#=
function VEGAS_rounding_map_OLD!(draw1, draw2, X, N_g, VEGAS_grid, VEGAS_spacing_grid) 
    draw1[1] = round(Int64, floor(X*N_g))
    #println(draw[1])
    draw2[1] = VEGAS_grid[draw1[1]+1]
    #println(draw[2])
    if (draw[1] != N_g)
        draw2[1] += VEGAS_spacing_grid[draw1[1]+1]*(X*N_g - draw1[1])
    end
end
=#

x (generic function with 1 method)

In [12]:
@time x(1, N_g, VEGAS_grid, VEGAS_spacing_grid)

  0.000004 seconds (1 allocation: 32 bytes)


(10.0, 11)

The Jacobian for this transformation is easily computed as:

$$
J(X) = N_g \Delta x_{i(X)}  ≡ J_{i(X)}
$$

along each axis. This is a step function, and the values are determined by the interval widths $ \Delta x_{i} $.

In [5]:
#=
function MC_sampling_OLD(x0, y0, z0, N_ev, N_g, VEGAS_grid, VEGAS_spacing_grid)
    
    draw = Array{Int}(undef, 3);
    VEGAS_draw_1 = Array{Int}(undef, 1);
    VEGAS_draw_2 = Array{Float64}(undef, 1);
    draw_float_sample = Array{Float64}(undef, 1);
    Uniform_distribution = Uniform(0, 1);
    
    result = 0.0
   
    for i=1:N_ev
        
          for i = 1:3
              # this is X \in [0,1]
              rand!(Uniform_distribution, draw_float_sample)
              #println(draw_float_sample[1])
            
              VEGAS_rounding_map_OLD!(VEGAS_draw_1, VEGAS_draw_2, draw_float_sample[1], N_g, VEGAS_grid, VEGAS_spacing_grid)

              draw[i] = round(Int, VEGAS_draw_2[1])
          end
    
        result += exp(-(1/2)*(draw[1] - x0)^2)*exp(-(1/2)*(draw[2] - y0)^2)*exp(-(1/2)*(draw[3] - z0)^2)*sin(draw[1]*draw[2])*cos(draw[3])
        
    end
    
    result = result*11^3/N_ev
        
    return result
    
end
=#

In [6]:
#@time MC_sampling_OLD(2, 6, 10, N_ev, N_g, VEGAS_grid, VEGAS_spacing_grid)

### Refining the grid

Given some samples $[X, Y, Z]_1 \dots [X, Y, Z]_N$ and the corresponding Monte Carlo estimate $\eqref{MC_estimator}$, now I want to estimate the following quantities:

$$
d^x_0 = \frac{1}{n^x_0} \sum_{x(X) \in \Delta x_0} \left( J \cdot f(x_0, y_0, z_0, [x(X), y(Y), z(Z)]) \right)^2
$$

$$
d^x_1 = \frac{1}{n^x_1} \sum_{x(X) \in \Delta x_1} \left( J \cdot f(x_0, y_0, z_0, [x(X), y(Y), z(Z)]) \right)^2
$$

$$
\dots
$$

$$
d^x_{N_g-1} = \frac{1}{n^x_{N_g-1}} \sum_{x(X) \in \Delta x_{N_g-1}}  \left( J \cdot f(x_0, y_0, z_0, [x(X), y(Y), z(Z)]) \right)^2
$$

etc. $\textbf{along each axis}$, where $n^x_i \approx \frac{N_{ev}}{N_g}$ is the number of samples in interval $\Delta x_i$.

Let's define a matrix:

\begin{align}
\text{VEGAS_d}= 
\begin{pmatrix}
d_0^x & d_1^x & \dots & d_{N_g-1}^x \\
d_0^y & d_1^y & \dots & d_{N_g-1}^y  \\
d_0^z & d_1^z & \dots & d_{N_g-1}^z  \\
\end{pmatrix}
\end{align}

It is also useful to consider a matrix to take into account the multeplicity:

\begin{align}
\text{VEGAS_d_multeplicity}= 
\begin{pmatrix}
n_0^x & n_1^x & \dots & n_{N_g-1}^x \\
n_0^y & n_1^y & \dots & n_{N_g-1}^y  \\
n_0^z & n_1^z & \dots & n_{N_g-1}^z  \\
\end{pmatrix}
\end{align}

Now we can implement the classic VEGAS algorithm. 

$\textbf{From now on, I stop describing explicitly all steps since it takes too much time}$

In [27]:
VEGAS_d = zeros(3, N_g);

Result_estimate = Array{Float64}(undef, 1);

In [28]:
function VEGAS_sampling(
    x0,
    y0,
    z0,
    N_ev,
    N_g,
    VEGAS_grid,
    VEGAS_spacing_grid,
    VEGAS_d,
    #VEGAS_d_multeplicity,
    V,
    Result_estimate,
)

    draw = Array{Int}(undef, 3)
    draw_float_sample = Array{Float64}(undef, 1)
    Uniform_distribution = Uniform(0, 1)

    VEGAS_index_draw = Array{Int}(undef, 3)
    VEGAS_d_multeplicity = zeros(3, N_g)
    Jacobians = Array{Float64}(undef, 3)
    VEGAS_index_draw = Array{Int}(undef, 3)

    total_result = 0.0

    for n = 1:N_ev

        for axis = 1:3
            # this is X in [0,1]
            rand!(Uniform_distribution, draw_float_sample)
            # println("The following number X was generated along axis $(axis) between 0 and 1:")
            # println(draw_float_sample[1])

            i_X = Int(floor(draw_float_sample[1] * N_g))
            delta_X = draw_float_sample[1] * N_g - i_X
            VEGAS_index_draw[axis] = i_X + 1

            # this is x(X) in [a,b]        
            if (i_X == N_g)
                draw_float_sample[1] = VEGAS_grid[VEGAS_index_draw[axis], axis]
            else
                draw_float_sample[1] =
                    VEGAS_grid[VEGAS_index_draw[axis], axis] +
                    VEGAS_spacing_grid[VEGAS_index_draw[axis], axis] * delta_X
            end

            #   println(
            #       "The following number x in the grid along axis $(axis) between 0 and 10 is the following",
            #   )
            #  println(draw_float_sample[1])

            #  println(
            #      "It corresponds to index $(i_X) in the grid, which means index $(VEGAS_index_draw[axis]) in the matrix",
            #  )

            Jacobians[axis] = N_g * VEGAS_spacing_grid[VEGAS_index_draw[axis], axis]
            #   println("This is the Jacobian along axis $(i):")
            #   println(Jacobians[i])

            VEGAS_d_multeplicity[axis, VEGAS_index_draw[axis]] += 1
            #println("This is the multeplicity of VEGAS_d:")
            #println(VEGAS_d_multeplicity)

            # we need to round to integers 
            draw_float_sample[1] = round(Int64, draw_float_sample[1])
            draw[axis] = round(Int64, draw_float_sample[1])
            #  println("The number was rounded to:")
            #  println(draw[axis])
            #   println(" ")
        end

        # println(" ")
        # println(" ")
        # println(" ")

        result =
            exp(-(1 / 2) * (draw[1] - x0)^2) *
            exp(-(1 / 2) * (draw[2] - y0)^2) *
            exp(-(1 / 2) * (draw[3] - z0)^2) *
            sin(draw[1] * draw[2]) *
            cos(draw[3]) *
            prod(Jacobians)

        total_result += result

        d_result = result^2

        for axis = 1:3
            VEGAS_d[axis, VEGAS_index_draw[axis]] += d_result
        end

    end

    # final normalization
    for axis = 1:3
        for j = 1:N_g
            VEGAS_d[axis, j] = VEGAS_d[axis, j] / VEGAS_d_multeplicity[axis, j]
        end
    end
    #VEGAS_d[:] .= VEGAS_d[:] ./ VEGAS_d_multeplicity[:]

    total_result *= (V / 10^3) / N_ev

    Result_estimate[1] = total_result

end


VEGAS_sampling (generic function with 1 method)

In [30]:
@time VEGAS_sampling(
    2,
    6,
    10,
    10^7,
    N_g,
    VEGAS_grid,
    VEGAS_spacing_grid,
    VEGAS_d,
    #VEGAS_d_multeplicity,
    V,
    Result_estimate,
)

  1.022456 seconds (8 allocations: 1.062 KiB)


0.6548089684771637

In [31]:
Result_estimate

1-element Vector{Float64}:
 0.6548089684771637

In [32]:
VEGAS_d[1, :]

25-element Vector{Float64}:
    0.0
 1105.0810810887303
 1470.6659673261015
 2389.761184213648
 5025.972665657154
 5199.973246303518
 3123.551088836936
 2429.8067959795703
 1831.5760122151582
  142.2873657286985
  136.52273051456726
   35.93874344376768
    0.8680322139218738
    0.6555995044018772
    0.0012444277497052456
    0.0012108119403256682
    0.0003080989852559667
    1.1394846649169965e-7
    8.644449026436092e-8
    1.4041542340347224e-12
    1.4107918070275705e-12
    3.4285569814101634e-13
    2.0548121477544333e-18
    1.574707407443367e-18
    3.8686421808090697e-25

In [33]:
function VEGAS_d_smoothing(N_g, VEGAS_d)

    VEGAS_sum_all_d = vec(sum(VEGAS_d, dims = 1))

    for i = 1:3

        VEGAS_d[1, i] = (7 * VEGAS_d[1, i] + VEGAS_d[2, i]) / (8 * VEGAS_sum_all_d[i])

        for n = 2:(N_g-1)
            VEGAS_d[n, i] =
                (VEGAS_d[n-1, i] + 6 * VEGAS_d[n, i] + VEGAS_d[n+1, i]) /
                (8 * VEGAS_sum_all_d[i])
        end

        VEGAS_d[N_g, i] =
            (VEGAS_d[N_g-1, i] + 7 * VEGAS_d[N_g, i]) / (8 * VEGAS_sum_all_d[i])
    end

end

VEGAS_d_smoothing (generic function with 1 method)

In [34]:
function VEGAS_d_compression(N_g, VEGAS_d, alpha)

    for i = 1:3
        for n = 1:N_g
            VEGAS_d[n, i] = ((1 - VEGAS_d[n, i]) / (log(1 / VEGAS_d[n, i])))^(alpha)
        end
    end

end

VEGAS_d_compression (generic function with 1 method)

In [35]:
function VEGAS_grid_refinement(
    N_g,
    VEGAS_d,
    VEGAS_grid,
    VEGAS_grid_improved,
    VEGAS_spacing_grid,
    VEGAS_spacing_grid_improved,
)

    delta_d = Array{Float64}(undef, 3)

    VEGAS_sum_all_d = vec(sum(VEGAS_d, dims = 1))

    #println(VEGAS_sum_all_d)

    delta_d[:] .= VEGAS_sum_all_d[:] ./ N_g

    for axis = 1:3

        VEGAS_grid_improved[1, axis] = VEGAS_grid[1, axis]
        VEGAS_grid_improved[end, axis] = VEGAS_grid[end, axis]

        # index of current improved x
        i = 0

        # index of current old x
        j = 0

        # amount of d accumulated
        S_d = 0

        while (true)

            i += 1

            # CHECK INDEX AT THE END
            if (i == N_g)
                break
            end

            while (S_d < delta_d[axis])

                S_d += VEGAS_d[j+1, axis]

                #  println(S_d)
                #  println(delta_d[axis])
                #  println(j)

                j += 1

            end

            S_d -= delta_d[axis]

            VEGAS_grid_improved[i+1, axis] =
                VEGAS_grid[j+1, axis] -
                (S_d / VEGAS_d[j, axis]) * VEGAS_spacing_grid[j, axis]

        end

        for j = 1:N_g
            VEGAS_spacing_grid_improved[j, axis] =
                VEGAS_grid_improved[j+1, axis] - VEGAS_grid_improved[j, axis]
        end

    end

end

VEGAS_grid_refinement (generic function with 1 method)

Notice that it is better to reshape the matrix as:

\begin{align}
\text{VEGAS_d} \longrightarrow
\begin{pmatrix}
d_0^x & d_0^y & d_0^z \\
d_1^x & d_1^y & d_1^z \\
\vdots & \vdots & \vdots \\
d_{N_g-1}^x & d_{N_g-1}^y & d_{N_g-1}^z \\
\end{pmatrix}
\end{align}

according to julia's convention for data storage

In [36]:
VEGAS_d = permutedims(VEGAS_d, (2, 1))

VEGAS_d_smoothing(N_g, VEGAS_d)

VEGAS_d_compression(N_g, VEGAS_d, 1.5)

VEGAS_grid_refinement(
    N_g,
    VEGAS_d,
    VEGAS_grid,
    VEGAS_grid_improved,
    VEGAS_spacing_grid,
    VEGAS_spacing_grid_improved,
)

VEGAS_d = permutedims(VEGAS_d, (2, 1))

3×25 Matrix{Float64}:
 0.0857793   0.169692    0.194852    …  0.0027998  0.00272011  0.00198159
 0.00656987  0.00732319  0.0105731      0.0296409  0.0280308   0.0141308
 0.0012737   0.00131978  0.00171771     0.320913   0.387738    0.506352

In [37]:
VEGAS_d

3×25 Matrix{Float64}:
 0.0857793   0.169692    0.194852    …  0.0027998  0.00272011  0.00198159
 0.00656987  0.00732319  0.0105731      0.0296409  0.0280308   0.0141308
 0.0012737   0.00131978  0.00171771     0.320913   0.387738    0.506352

In [38]:
VEGAS_grid_improved

26×3 Matrix{Float64}:
  0.0        0.0       0.0
  0.4266     2.94438   5.90162
  0.6554     3.67007   7.02813
  0.873328   4.13093   7.84706
  1.07258    4.46855   8.19487
  1.25575    4.68971   8.41784
  1.41037    4.89832   8.52476
  1.56499    5.09445   8.63167
  1.6874     5.28168   8.73859
  1.80038    5.45854   8.83924
  1.91337    5.62999   8.93144
  2.02679    5.77977   9.02364
  2.14163    5.92955   9.11584
  2.25648    6.07912   9.20665
  2.37132    6.22851   9.28296
  2.511      6.3779    9.35927
  2.65895    6.53031   9.43558
  2.8078     6.68325   9.51189
  2.97527    6.83678   9.5882
  3.14274    6.99221   9.6494
  3.33155    7.14765   9.70783
  3.53145    7.32443   9.76627
  3.90673    7.51205   9.8247
  4.38899    7.90632   9.88313
  5.30978    8.53397   9.94157
 10.0       10.0      10.0

In [202]:
VEGAS_spacing_grid_improved

25×3 Matrix{Float64}:
 0.336545  0.835393  0.980221
 0.285983  0.749243  0.930431
 0.267072  0.688617  0.854636
 0.253935  0.600453  0.782713
 0.230393  0.48125   0.737113
 0.223662  0.417209  0.66083
 0.1961    0.374785  0.580321
 0.1958    0.331234  0.537165
 0.200743  0.284916  0.469047
 0.201023  0.271156  0.439551
 0.222721  0.263529  0.424139
 0.22624   0.254583  0.390244
 0.238741  0.241135  0.315877
 0.25163   0.238076  0.245034
 0.264884  0.238313  0.190232
 0.358285  0.238831  0.175224
 0.375173  0.239453  0.167305
 0.420171  0.243016  0.159456
 0.511116  0.248327  0.159456
 0.582611  0.273998  0.14285
 0.65856   0.365645  0.142745
 0.736012  0.423178  0.13941
 0.839734  0.470873  0.125332
 0.912933  0.570093  0.125332
 1.00993   0.656692  0.125332

## Full VEGAS algorithm

Let's try now to write the full algorithm 

Let's try now to write the full algorithm 

The list of parameters is the following:

- $x_0$, $y_0$ $z_0$: Input numbers of function $F$
- $N_g$: Number of elements in the grid
- $N_{ev}$: Number of Monte Carlo sample in the grid for each iteration
- $N_{it}$: Number of VEGAS iterations (at each iteration the grid improves)
- $\alpha$: Parameter used in the compression phase
- $V$: Number of total configurations (used to re-normalize in the discrete case)

In [273]:
function VEGAS_algorithm(x0, y0, z0, N_g, N_ev, N_it, alpha, V)


    VEGAS_grid = Array{Float64}(undef, N_g + 1, 3)
    VEGAS_grid_improved = Array{Float64}(undef, N_g + 1, 3)

    a = 0
    b = 10
    for i = 1:3
        VEGAS_grid[:, i] .= collect(LinRange(a, b, N_g + 1))
    end

    VEGAS_spacing_grid = Array{Float64}(undef, N_g, 3)
    VEGAS_spacing_grid_improved = Array{Float64}(undef, N_g, 3)

    for i = 1:3
        for j = 1:N_g
            VEGAS_spacing_grid[j, i] = VEGAS_grid[j+1, i] - VEGAS_grid[j, i]
        end
    end

    Result_estimate = Array{Float64}(undef, 1)

    for n = 1:N_it

        println("VEGAS iteration $(n)")
        flush(stdout)

        println("The current grid along first axis is $(VEGAS_grid[:,1])")
        flush(stdout)

        VEGAS_d = zeros(3, N_g)

        # populates VEGAS_d and Result_estimate by using current VEGAS_grid and VEGAS_spacing_grid
        VEGAS_sampling(
            x0,
            y0,
            z0,
            N_ev,
            N_g,
            VEGAS_grid,
            VEGAS_spacing_grid,
            VEGAS_d,
            V,
            Result_estimate,
        )

        println("The current estimate of the sum is $(Result_estimate[1])")
        flush(stdout)

        VEGAS_d = permutedims(VEGAS_d, (2, 1))

        VEGAS_d_smoothing(N_g, VEGAS_d)

        VEGAS_d_compression(N_g, VEGAS_d, alpha)

        # refines the grid by using VEGAS_d and the current VEGAS_grid and VEGAS_spacing_grid
        VEGAS_grid_refinement(
            N_g,
            VEGAS_d,
            VEGAS_grid,
            VEGAS_grid_improved,
            VEGAS_spacing_grid,
            VEGAS_spacing_grid_improved,
        )

        VEGAS_d = permutedims(VEGAS_d, (2, 1))

        #println(VEGAS_spacing_grid[:, 2])

        VEGAS_spacing_grid[:] .= VEGAS_spacing_grid_improved[:]

        VEGAS_grid[:] .= VEGAS_grid_improved[:]

    end

end

VEGAS_algorithm (generic function with 1 method)

In [274]:
VEGAS_algorithm(2, 6, 10, 25, 10^7, 10, 1.5, V)

VEGAS iteration 1
The current grid along first axis is [0.0, 0.4, 0.8, 1.2, 1.6, 2.0, 2.4, 2.8000000000000003, 3.2, 3.5999999999999996, 4.0, 4.4, 4.8, 5.2, 5.6000000000000005, 6.0, 6.4, 6.800000000000001, 7.199999999999999, 7.6, 8.0, 8.4, 8.8, 9.200000000000001, 9.6, 10.0]
The current estimate of the sum is 0.6205778274187709
VEGAS iteration 2
The current grid along first axis is [0.0, 0.426981591864192, 0.6563566928150318, 0.8747565810747882, 1.0747675405748005, 1.257851994432361, 1.4125894243598074, 1.5673268542872538, 1.688609506673692, 1.80093726428139, 1.9132650218890879, 2.0262863736193957, 2.1416583442439885, 2.2570303148685813, 2.372402285493174, 2.512488656937732, 2.66034565508087, 2.809295362914235, 2.976849009230441, 3.144402655546647, 3.3332598980499624, 3.5326964769157883, 3.910731229268739, 4.389470433194461, 5.310249392376203, 10.0]
The current estimate of the sum is 0.6460667148443591
VEGAS iteration 3
The current grid along first axis is [0.0, 0.522812885767573, 0.7263